In [17]:
import numpy as np 
import pandas as pd
from scipy.optimize import minimize  

In [10]:
arg_source = pd.read_csv('D:/jupyter_dir/yancao/data/step1.csv', sep='\t')
arg_source.head()

,Unnamed: 0,item,price,itemid,sold,cap
0,1,中南海(京韵细支烤烟),75000,6.901030e+12,14,42
1,2,云烟(中支金腰带),75000,6.901030e+12,183,243
2,3,云烟(细支大重九),250000,6.901030e+12,153,186
3,4,云烟(细支珍品),65000,6.901030e+12,474,580
4,5,云烟(软大重九),245000,6.901030e+12,80,122


In [11]:
# 参数和决策变量
price = arg_source['price'].values
sold = arg_source['sold'].values 
cap = arg_source['cap'].values

sold_obj = 2800
price_obj = 40000
C = 1

$
min f = C*(Sold\_obj - \sum_{i=1}^I Sold\_next_i)^2 + \sum_{i=1}^I [(Sold\_next_i - Sold_i)/Sold_i]^2
$

$s.t.$

$
0 <= Sold\_next_i <= Cap_i
$

$
Sold\_obj*Price\_obj*0.9 <= \sum_{i=1}^I Price_i*Sold\_next_i <= Sold\_obj*Price\_obj*1.1
$

In [12]:
# 目标函数
obj = lambda x: ((sold_obj*250 - x.sum())**2)*C + (((x - sold)/sold)**2).sum()

# 约束条件
cons = []
cons.append({'type':'ineq', 'fun': lambda x: (x*price).sum() - sold_obj*price_obj*0.9})
cons.append({'type':'ineq', 'fun': lambda x: sold_obj*price_obj*1.1 - (x*price).sum()})
for i in range(sold.size):
    cons.append({'type':'ineq', 'fun': lambda x: x[i]})
    cons.append({'type':'ineq', 'fun': lambda x: cap[i] - x[i]})

In [13]:
# 求解
sold_next = sold
sol = minimize(obj, sold_next, method='TNC', constraints=cons)

print(sol.fun)
print(sol.success)
# print(sol.x)

1.151835913968868
True
D:\installation\anaconda\lib\site-packages\scipy\optimize\_minimize.py:520: RuntimeWarning: Method TNC cannot handle constraints.
  RuntimeWarning)


In [143]:
# 检验约束条件
check = lambda x: x['fun'](sol.x) >= 0
np.array(list(map(check, cons)))

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [145]:
arg_source['sold_next'] = pd.Series(np.round(sol.x), dtype='int')
arg_source

,Unnamed: 0,item,price,itemid,sold,cap,sold_next
0,1,中南海(京韵细支烤烟),75000,6.901030e+12,14,42,14
1,2,云烟(中支金腰带),75000,6.901030e+12,183,243,183
2,3,云烟(细支大重九),250000,6.901030e+12,153,186,153
3,4,云烟(细支珍品),65000,6.901030e+12,474,580,476
4,5,云烟(软大重九),245000,6.901030e+12,80,122,80
5,6,云烟(软珍品),60000,6.901030e+12,4695,5643,4855
6,7,冬虫夏草(和润),125000,6.901030e+12,522,657,524
7,8,凤凰(细支),87500,6.901030e+12,594,728,597
8,9,利群(蓝天),40000,6.901030e+12,8018,9654,8483
9,10,利群(西子阳光),77500,6.901030e+12,714,870,718
